In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Thesis

/content/drive/MyDrive/Thesis


In [3]:
# !git clone -b v0.10.2 https://github.com/pytorch/fairseq
!pip3 install ./fairseq/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fairseq
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92 kB 587 kB/s 
     |████████████████████████████████| 151 kB 8.9 MB/s 
     |████████████████████████████████| 117 kB 63.5 MB/s 
     |████████████████████████████████| 79 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
  Created wheel for fairseq: 

In [4]:
%cd contextual-mt/

/content/drive/MyDrive/Thesis/contextual-mt


In [5]:
!pip3 install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Thesis/contextual-mt
  Running setup.py develop for contextual-mt


In [6]:
model_ckpt_p00="/content/drive/MyDrive/Thesis/contextual-mt/checkpoint0"
model_ckpt_p01="/content/drive/MyDrive/Thesis/contextual-mt/checkpoint"
source_context_size=0
target_context_size=1
source_lang="en"
target_lang="vi"

In [7]:
!pip3 install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.8 MB/s 


In [8]:
import os
import sentencepiece as sp
import contextual_mt
from fairseq import utils, hub_utils

package_p00 = hub_utils.from_pretrained(
    model_ckpt_p00, checkpoint_file="checkpoint_best.pt"
)

package_p01 = hub_utils.from_pretrained(
    model_ckpt_p01, checkpoint_file="checkpoint_best.pt"
)

models_p00 = package_p00["models"]
for model_p00 in models_p00:
    model_p00.cuda()
    model_p00.eval()

models_p01 = package_p01["models"]
for model_p01 in models_p01:
    model_p01.cuda()
    model_p01.eval()


# load dict, params and generator from task
src_dict_p00 = package_p00["task"].src_dict
tgt_dict_p00 = package_p00["task"].tgt_dict

src_dict_p01 = package_p01["task"].src_dict
tgt_dict_p01 = package_p01["task"].tgt_dict
# load sentencepiece models (assumes they are in the checkpoint dirs)
# FIXME: is there someway to have it in `package`
if os.path.exists(os.path.join(model_ckpt_p00, "spm.model")):
    spm_p00 = sp.SentencePieceProcessor()
    spm_p00.Load(os.path.join(model_ckpt_p00, "spm.model"))
    src_spm_p00 = spm_p00
    tgt_spm_p00 = spm_p00
else:
    src_spm_p00 = sp.SentencePieceProcessor()
    src_spm_p00.Load(os.path.join(model_ckpt_p00, f"spm.{source_lang}.model"))
    tgt_spm_p00 = sp.SentencePieceProcessor()
    tgt_spm_p00.Load(os.path.join(model_ckpt_p00, f"spm.{target_lang}.model"))


if os.path.exists(os.path.join(model_ckpt_p01, "spm.model")):
    spm_p01 = sp.SentencePieceProcessor()
    spm_p01.Load(os.path.join(model_ckpt_p01, "spm.model"))
    src_spm_p01 = spm_p01
    tgt_spm_p01 = spm_p01
else:
    src_spm_p01 = sp.SentencePieceProcessor()
    src_spm_p01.Load(os.path.join(model_ckpt_p01, f"spm.{source_lang}.model"))
    tgt_spm_p01 = sp.SentencePieceProcessor()
    tgt_spm_p01.Load(os.path.join(model_ckpt_p01, f"spm.{target_lang}.model"))

In [81]:
source_file="/content/drive/MyDrive/Thesis/contextual-mt/projects/tir1/corpora/dialogue_mt/iwslt2015/en-vi/test.en-vi.en"
target_file="/content/drive/MyDrive/Thesis/contextual-mt/projects/tir1/corpora/dialogue_mt/iwslt2015/en-vi/test.en-vi.vi"
docids_file="/content/drive/MyDrive/Thesis/contextual-mt/projects/tir1/corpora/dialogue_mt/iwslt2015/en-vi/test.en-vi.docids"
hyps_file="/content/drive/MyDrive/Thesis/contextual-mt/predictions/test.pred.tgt"
hyps_file0="/content/drive/MyDrive/Thesis/contextual-mt/predictions/test.pred.tgt0"
batch_size=8

In [82]:
from contextual_mt.utils import parse_documents
from contextual_mt.docmt_cxmi import compute_cxmi
import numpy as np

# load files needed
with open(source_file, "r") as src_f:
    srcs = [line.strip() for line in src_f]
with open(docids_file, "r") as docids_f:
    docids = [int(idx) for idx in docids_f]
with open(target_file, "r") as tgt_f:
    refs = [line.strip() for line in tgt_f]
with open(hyps_file, "r") as hyp_f:
    hyps = [line.strip() for line in hyp_f]
with open(hyps_file0, "r") as hyp_f0:
    hyps0 = [line.strip() for line in hyp_f0]
documents = parse_documents(srcs, refs, docids)

In [83]:
sample_cxmis_p00, ids_p00 = compute_cxmi(
        documents,
        models_p00,
        src_spm_p00,
        src_dict_p00,
        tgt_spm_p00,
        tgt_dict_p00,
        source_context_size,
        target_context_size,
        batch_size=batch_size
)

In [84]:
sample_cxmis_p01, ids_p01 = compute_cxmi(
        documents,
        models_p01,
        src_spm_p01,
        src_dict_p01,
        tgt_spm_p01,
        tgt_dict_p01,
        source_context_size,
        target_context_size,
        batch_size=batch_size
)

In [85]:
print(np.mean(sample_cxmis_p00))

0.012680513


In [86]:
print(np.mean(sample_cxmis_p01))

0.0136776855


In [87]:
import pandas as pd

In [88]:
cxmis_p00 = []
cxmis_p01 = []
for cxmi in sample_cxmis_p00:
    cxmis_p00.append(cxmi.numpy().tolist())

for cxmi in sample_cxmis_p01:
    cxmis_p01.append(cxmi.numpy().tolist())

In [97]:
import nltk
bleu_p00 = []
bleu_p01 = []
for i in range(len(refs)):
    bleu_p00.append(nltk.translate.bleu_score.sentence_bleu([refs[i]], hyps0[i])*100)
    bleu_p01.append(nltk.translate.bleu_score.sentence_bleu([refs[i]], hyps[i])*100)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [105]:
df = pd.DataFrame(list(zip(cxmis_p00, cxmis_p01, bleu_p00, bleu_p01, srcs, refs, hyps0, hyps)), columns=['cxmi_p00', 'cxmi_p01','bleu_p00', 'bleu_p01', 'srcs', 'refs', 'hyps_p00', 'hyps_p01'
                                                                                     ])

In [106]:
df['delta_cxmi'] = df['cxmi_p01'] - df['cxmi_p00']
df['delta_bleu'] = df['bleu_p01'] - df['bleu_p00']

In [111]:
df.to_csv("/content/drive/MyDrive/Thesis/example.csv")

In [109]:
df[['srcs', 'refs', 'hyps_p00', 'hyps_p01', 'delta_cxmi', 'delta_bleu']]

,srcs,refs,hyps_p00,hyps_p01,delta_cxmi,delta_bleu
0,"When I was little, I thought my country was th...","Khi tôi còn nhỏ, Tôi nghĩ rằng BắcTriều_Tiên l...","Khi còn nhỏ, tôi nghĩ đất_nước tôi là nước tốt...","Khi tôi còn nhỏ, tôi nghĩ rằng đất_nước mình l...",0.000000,4.025259
1,And I was very proud.,Tôi đã rất tự_hào về đất_nước tôi.,Và tôi rất tự_hào.,Và tôi rất tự_hào.,0.000000,0.000000
2,"In school, we spent a lot of time studying the...","Ở trường, chúng_tôi dành rất nhiều thời_gian đ...","Ở trường_học, chúng_tôi dành rất nhiều thời_gi...","Ở trường, chúng_tôi đã dành rất nhiều thời_gia...",0.000000,5.570202
3,Although I often wondered about the outside wo...,Mặc_dù tôi đã từng tự hỏi không biết thế_giới ...,Mặc_dù tôi thường băn_khoăn về thế_giới bên ng...,Mặc_dù tôi thường thắc_mắc về thế_giới bên ngo...,0.000000,1.245602
4,"When I was seven years old, I saw my first pub...","Khi tôi lên 7, tôi chứng_kiến cảnh người ta xử...","Khi tôi 7 tuổi, tôi nhìn thấy cuộc hành_quyết ...","Khi tôi bảy tuổi, tôi thấy cuộc hành_quyết côn...",0.000000,-6.468584
...,...,...,...,...,...,...
1263,"I truly believe, if we can see one another as ...","Tôi thực_sự tin, nếu ta coi người khác như nhữ...","Tôi thực_sự tin rằng, nếu chúng_ta có_thể nhìn...","Tôi thực_sự tin rằng, nếu chúng_ta có_thể nhìn...",0.042511,8.726223
1264,These images are not of issues. They are of pe...,Những tấm hình không phải là về bản_thân vấnđề...,Những hình_ảnh này không phải là vấn_đề của vấ...,Những hình_ảnh này không phải là vấn_đề. Chúng...,-0.000594,11.958713
1265,There is not a day that goes by that I don't t...,Không có ngày nào mà tôi không nghĩ về những n...,Không có một ngày trôi qua mà tôi không nghĩ đ...,Không có một ngày nào được chứng_minh là tôi k...,0.014627,-8.512379
1266,I hope that these images awaken a force in tho...,Tôi hi_vọng những tấm hình sẽ đánh_thức một ng...,Tôi hi_vọng rằng những hình_ảnh đó bị đánh_thứ...,Tôi hy_vọng những hình_ảnh này đánh_thức một l...,-0.032932,5.119625


In [ ]:
df.to_csv("/content/drive/MyDrive/Thesis/example")

In [80]:
# !bash inference.sh

  0% 0/1268 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
100% 1268/126